<a href="https://colab.research.google.com/github/4GeeksAcademy/Proyecto_Final_Close-Loop-Intelligence/blob/main/Proyecto_Final_Close_Loop_Intelligence_pynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Idea Integral Ganadora: Close Loop Intelligence
---

1. ***Definición del Problema (El "Pitch" de Negocio)***

    Muchas empresas sufren de dos males: el stockout (perder ventas por no tener producto) o el exceso de inventario (dinero muerto en estanterías que se deprecia). Este proyecto construye un Sistema de Inteligencia de Ciclo Cerrado que predice cuánto se va a vender para decidir exactamente cuánto comprar y qué promocionar para salir del inventario lento.

2. **Arquitectura de Datos (Paso 2 y 3: Ingesta y SQL)**

    Para llegar a las 60,000+ filas, utilizaremos un dataset transaccional histórico.

    Origen: Dataset de Retail (tipo Online Retail II o Instacart).

    Modelo de Datos (SQL): Almacenarás los datos en una base de datos relacional con este esquema:

        Tabla_Ventas: ID_Transaccion, Fecha, ID_Producto, Cantidad, Precio_Unitario, ID_Cliente.

        Tabla_Productos: ID_Producto, Categoria, Costo_Reposicion, Stock_Actual, Punto_Reorden.

    Uso de SQL (Crítico): No solo cargarás los datos. Usarás SQL para crear una Vista Maestra mediante un JOIN. Calcularás el "Lead Time" (tiempo de entrega) y la "Venta Promedio Diaria" usando GROUP BY y funciones agregadas. Esta vista será la que alimentará tu modelo de Machine Learning.

3. **El Cerebro del Proyecto (Paso 4, 5 y 6: EDA y ML)**

    Aquí es donde ocurre la magia. Dividiremos el modelo en dos enfoques:

    *Enfoque de Ventas (Predicción de Demanda):*

    Utilizarás un modelo de Series Temporales o Regresión (como Random Forest Regressor) para predecir cuántas unidades de un producto se venderán en los próximos 15-30 días basándote en la estacionalidad y tendencias históricas.

    *Enfoque de Compras/Inventario (Optimización):*

    Clasificación ABC: Usarás los datos para clasificar productos. Los "A" son los que más dinero generan (nunca pueden faltar), los "C" son los lentos.

    Cálculo de Stock de Seguridad: Aplicarás fórmulas estadísticas para determinar el inventario mínimo necesario.

4. **La Salida en Ventas: "Revenue Optimizer"**

    Para darle ese toque final de ventas que mencionaste, añadiremos un componente de Estrategia Comercial:

    Si el modelo predice una demanda baja para un producto con mucho stock (Exceso), la aplicación sugerirá automáticamente un descuento o promoción de "cross-selling" para forzar la salida de esa mercancía y recuperar flujo de caja.

5. **Desarrollo de la Web App (Paso 7: La interfaz)**
    
    La aplicación (en Streamlit) tendrá tres pestañas claras para el usuario final:

    *Dashboard Ejecutivo (Ventas):* Gráficos elocuentes que muestran la proyección de ingresos para el próximo mes y el "Top 10" de productos estrella.

    *Módulo de Compras (Reposición):* Una tabla generada por tu modelo que diga: "Debes pedir 500 unidades del Producto X hoy mismo al proveedor Y para no quedar desabastecido el lunes".

    *Módulo de Ofertas (Salida de Inventario):* Una sección que resalte: "¡Alerta! Tienes demasiado stock de este producto. Aplica un 15% de descuento para incentivar la venta".

In [ ]:
#Realizamos la instalacion para ejecutar la conexion con Kaggle
!pip install kaggle
!pip install opendatasets

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import opendatasets as od

od.download('https://www.kaggle.com/datasets/olistbr/brazilian-ecommerce')